In [44]:
from langchain_groq import ChatGroq

In [48]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

In [70]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-46197?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Floor Manager (Coach) – Amsterdam (Osdorp)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresFloor Manager (Coach) – Amsterdam (Osdorp)Amsterdam, North HollandBecom

In [82]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'[\n  {\n    "role": "Floor Manager (Coach)",\n    "experience": "Three years of retail and/or consumer service experience, One year of leadership experience",\n    "skills": "Demonstrated ability to recruit, build and lead high-performing teams, Enthusiasm for engaging with consumers and inspiring them to become Nike Members, Digital literacy—desire to learn and adapt to technology needs with agility",\n    "description": "The Nike Coach position is right up your alley if you love leading people and have a passion for retail. We\'re here to deliver a premium experience for our Nike teammates and our consumers, and a Coach plays a vital role in making this happen. Your understanding of store operations brings clarity to your team, and your ability to empower peers and direct reports gets results."\n  }\n]'

In [85]:
import json

json_res = json.loads(res.content)

if isinstance(json_res, list) and len(json_res) > 0:
    json_res = json_res[0]

print(json_res)
print(type(json_res))

{'role': 'Floor Manager (Coach)', 'experience': 'Three years of retail and/or consumer service experience, One year of leadership experience', 'skills': 'Demonstrated ability to recruit, build and lead high-performing teams, Enthusiasm for engaging with consumers and inspiring them to become Nike Members, Digital literacy—desire to learn and adapt to technology needs with agility', 'description': "The Nike Coach position is right up your alley if you love leading people and have a passion for retail. We're here to deliver a premium experience for our Nike teammates and our consumers, and a Coach plays a vital role in making this happen. Your understanding of store operations brings clarity to your team, and your ability to empower peers and direct reports gets results."}
<class 'dict'>


In [86]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [87]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [89]:
job = json_res

In [90]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [91]:
job

{'role': 'Floor Manager (Coach)',
 'experience': 'Three years of retail and/or consumer service experience, One year of leadership experience',
 'skills': 'Demonstrated ability to recruit, build and lead high-performing teams, Enthusiasm for engaging with consumers and inspiring them to become Nike Members, Digital literacy—desire to learn and adapt to technology needs with agility',
 'description': "The Nike Coach position is right up your alley if you love leading people and have a passion for retail. We're here to deliver a premium experience for our Nike teammates and our consumers, and a Coach plays a vital role in making this happen. Your understanding of store operations brings clarity to your team, and your ability to empower peers and direct reports gets results."}

In [92]:
job = json_res
job['skills']

'Demonstrated ability to recruit, build and lead high-performing teams, Enthusiasm for engaging with consumers and inspiring them to become Nike Members, Digital literacy—desire to learn and adapt to technology needs with agility'

In [93]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are a professional representing a company. Write a cold email to the client regarding the job mentioned above, 
        describing your company's ability to meet their requirements. Highlight your company's capabilities and experience, 
        and include examples from the provided portfolio links to demonstrate your expertise: {link_list}.
        
        Ensure the email is professional, concise, and directly addresses the client's needs. Avoid any preamble.
        ### EMAIL (NO PREAMBLE):
        """

)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expertise in Delivering High-Performing Teams for Nike Coach Position

Dear Hiring Manager,

We understand that finding the right candidate for the Floor Manager (Coach) role at Nike requires a unique blend of leadership skills, retail experience, and digital literacy. Our company specializes in providing top talent with a proven track record of success in driving high-performing teams and delivering exceptional consumer experiences.

With over three years of experience in retail and consumer service, our candidates possess the expertise to lead and inspire teams to achieve outstanding results. Our leadership development programs focus on empowering peers and direct reports, ensuring that our candidates can drive results and make a significant impact on store operations.

Our team's digital literacy and agility in adapting to technology needs are unparalleled. We have a proven track record of leveraging technology to enhance consumer experiences and drive business growth. For 